<a href="https://www.kaggle.com/code/kavishchaudhary1003/playground-s3-e5?scriptVersionId=119873045" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from tqdm import tqdm
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import cohen_kappa_score, accuracy_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier, IsolationForest, StackingClassifier, RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import StratifiedKFold

import os
for dirname, _, filenames in os.walk('/kaggle/input/playground-series-s3e5'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



# Competition Page
https://www.kaggle.com/competitions/playground-series-s3e5

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s3e5/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e5/test.csv')

In [ ]:
train.duplicated().sum()
train.isna().sum()

### Check for null and duplicates


In [ ]:
train.set_index('Id', inplace = True)
test.set_index('Id', inplace = True)

In [ ]:
train.describe()

### Data description

In [ ]:
corr = train.corr()
corr.style.background_gradient(cmap = 'coolwarm')

### Coorelation heat map

In [ ]:
If = IsolationForest()
If.fit(train)
train['anamoly'] = If.predict(train)
train.anamoly.value_counts()
train = train[train.anamoly ==1]
train.drop(columns =['anamoly'], inplace = True)

### Isolation forest to remove outliers

In [ ]:
y = train.quality
train.drop(columns = ['quality'], inplace = True)


### Separating features and target variable

In [ ]:
for i in train.columns:
    sns.boxplot(x = train[i])
    plt.show()

### Boxplot to check the data symmetry

In [ ]:
for i in train.columns:
    sns.histplot(train[i], kde = True)
    plt.show()

### Histogram to check the distribution

In [ ]:
cohen_kappa = make_scorer(cohen_kappa_score, weights = 'quadratic', greater_is_better = False)

### Setting the scorer as cohen_kappa_scorer

In [ ]:
def eval_model(model, param, train, y, cohen_kappa):
    outer_cv = KFold(n_splits = 5, shuffle = True, random_state = 0)
    inner_cv = KFold(n_splits = 5, shuffle = True, random_state = 0)
    random = RandomizedSearchCV(model, param, n_jobs = -1, cv= inner_cv, scoring = cohen_kappa, random_state = 0)
    scores = []
    n_iter = random.n_iter
    with tqdm(total = n_iter) as pbar:
        for i in range(n_iter):
            random.set_params(n_iter = 1)
            for train_index, test_index in cv_outer.split(train, y):
                trainx, testx = train.iloc[train_index], train.iloc[test_index]
                trainy, testy = y.iloc[train_index], y.iloc[test_index]
                random.fit(trainx, trainy)
                scores.append(random.best_estimator_.score(testx, testy))
            pbar.update()
    print("Average cohen_kappa", np.mean(scores))
    print(random.best_params_)

In [ ]:
model = DecisionTreeClassifier()
criterion = ['gini', 'entropy']
min_samples_split = np.array(range(2, 100))
max_depth = np.array(range(1, 100))
param = {'criterion': criterion, 'min_samples_split': min_samples_split, 'max_depth': max_depth}
eval_model(model, param, train, y, cohen_kappa)

### Decision Tree
### Randomizedsearch with nested cv
### Score of 0.578

In [ ]:
model = ExtraTreesClassifier()
criterion = ['gini', 'entropy']
max_depth = np.array(range(50, 150))
min_samples_split = np.array(range(1, 100))
n_estimators = np.array(range(100,300, 2))
param = {'criterion': criterion, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'n_estimators': n_estimators}
eval_model(model, param, train, y, cohen_kappa)

### Extra Trees
### Randomizedsearch with nested cv
### Score of 0.587

In [ ]:
model = BaggingClassifier()
n_estimators = np.array(range(100,300,2))
max_features = np.array(range(1,12))
param = {'n_estimators': n_estimators, 'max_features': max_features}
eval_model(model, param, train, y, cohen_kappa)

### Bagging Classifier
### Randomizedsearch with nested cv
### Score of 0.579

In [ ]:
model = AdaBoostClassifier()
n_estimators = np.array(range(100, 300, 2))
learning_rate = np.array(range(1, 20))
param = {'n_estimators': n_estimators, 'learning_rate': learning_rate}
eval_model(model, param, train, y, cohen_kappa)      

### Ada Boost
### Randomizedsearch with nested cv
### Score of 0.316

In [ ]:
le = LabelEncoder()
transformy = pd.DataFrame(le.fit_transform(y.values.ravel()))

In [ ]:
model = XGBClassifier(objective='multi:softmax',
                     eval_metric='mlogloss', num_class = 6)
learning_rate = np.linspace(0.01, 0.3, 10)
eta = np.linspace(0.01, 0.3, 10)
gamma = np.linspace(0, 1, 10)
max_depth = np.array(range(1, 10))
alpha= np.array(range(0, 50))
reg_lambda = np.linspace(0, 100, 10)
colsample_bytree = np.linspace(0.1,1, 10)
subsample = np.linspace(0.1, 1, 10)
min_child_weight = np.array(range(0,100,10))
n_estimators = np.array(range(200, 2000,100 ))
param = {'learning_rate': learning_rate, 'eta': eta, 'gamma': gamma, 'max_depth': max_depth, 'alpha': alpha,'reg_lambda': reg_lambda,
        'colsample_bytree': colsample_bytree, 'subsample': subsample, 'min_child_weight': min_child_weight, 'n_estimators': n_estimators}
cv_outer = KFold(n_splits = 5, shuffle = True, random_state = 0)
cv_inner = KFold(n_splits = 5, shuffle = True, random_state = 0)
scores = []
rxgb = RandomizedSearchCV(model, param, cv = cv_inner, n_jobs = -1, scoring = cohen_kappa)
n_iter = rxgb.n_iter
with tqdm(total = n_iter) as pbar:
    for i in range(n_iter):
        rxgb.set_params(n_iter = 1)
        for train_index, test_index in cv_outer.split(train, transformy):
            trainx, testx = train.iloc[train_index], train.iloc[test_index]
            trainy, testy = transformy.iloc[train_index], transformy.iloc[test_index]
            rxgb.fit(trainx, trainy)
            predy = rxgb.best_estimator_.predict(testx)
            scores.append(cohen_kappa_score(testy, predy))
        pbar.update()
print('Average score', np.mean(scores))
print(rxgb.best_params_)

In [ ]:
model = XGBClassifier(subsample = 0.4, reg_lambda = 44.44, n_estimators = 1200, min_child_weight = 50, max_depth = 8, learning_rate =0.1366, gamma = 0.4444, eta= 0.04222, colsample_bytree = 0.4, alpha = 44, random_state = 0)
model.fit(trainx, trainy)
predictions = model.predict(test)


In [ ]:
predictions = le.inverse_transform(predictions)
predictions = pd.DataFrame(predictions, index = test.index, columns =['quality'])
predictions.to_csv('submission.csv')

### XGB Boost
### Randomizedsearch with nested cv
### Score of 0.17

In [ ]:
level_0_estimators = dict()
level_0_estimators["logreg"] = LogisticRegression(max_iter = 100000, random_state = 100)
level_0_estimators["forest"] = model = RandomForestClassifier(criterion = 'gini', min_samples_split = 42, max_depth = 41, n_estimators=75, random_state = 100)
 
level_0_columns = [f"{name}_prediction" for name in level_0_estimators.keys()]
 
level_1_estimator =  RandomForestClassifier(n_estimators = 140, min_samples_split = 23, max_depth = 81, criterion = 'gini', random_state = 100)
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 100)
model = StackingClassifier(estimators=list(level_0_estimators.items()), 
                                    final_estimator=level_1_estimator, 
                                    passthrough=True, cv=kfold, stack_method="predict_proba")

model.fit(trainx, trainy.values.ravel())
predy = model.predict(testx)
print('score',cohen_kappa_score(testy,  predy, weights = "quadratic"))

### Stacking Classifier
### Randomizedsearch with nested cv
### Score of 0.49

In [ ]:
model = LGBMClassifier()
learning_rate = np.linspace(0.01, 0.2, 10)
max_depth = np.array(range(1, 10))
alpha= np.array(range(0, 50))
reg_lambda = np.linspace(0, 100, 10)
colsample_bytree = np.linspace(0.1,1, 10)
subsample = np.linspace(0.1, 1, 10)
subsample_freq = np.array(range(1, 10, 10))
min_child_samples  = np.array(range(1,50,10))
n_estimators = np.array(range(200, 2000,100 ))
num_leaves =np.array(range(2, 200, 11))
param = {'learning_rate': learning_rate, 'max_depth': max_depth, 'alpha': alpha, 'reg_lambda': reg_lambda, 'colsample_bytree': colsample_bytree,
        'subsample': subsample, 'subsample_freq': subsample_freq, 'min_child_samples': min_child_samples, 'n_estimators': n_estimators,
        'num_leaves': num_leaves}
cv_inner = KFold(n_splits = 5, shuffle = True, random_state = 0)
cv_outer = KFold(n_splits = 5, shuffle = True, random_state = 0)
scores = []
rlgbm = RandomizedSearchCV(model, param, cv= cv_inner, scoring = cohen_kappa, n_jobs = -1, random_state = 0)
n_iter = rlgbm.n_iter
with tqdm(total = n_iter) as pbar:
    for i in range(n_iter):
        rlgbm.set_params(n_iter = 1)
        for train_index, test_index in cv_outer.split(train, y):
            trainx, testx = train.iloc[train_index], train.iloc[test_index]
            trainy, testy = y.iloc[train_index], y.iloc[test_index]
            rlgbm.fit(trainx, trainy.values.ravel())
            predy = rlgbm.best_estimator_.predict(testx)
            scores.append(cohen_kappa_score(testy, predy))
        pbar.update()
print('Average Score', np.mean(scores))
print(rlgbm.best_params_)

### LGBM
### Randomizedsearch with nested cv
### Score of 0.245

In [ ]:
model = LGBMClassifier(subsample_freq = 1, subsample = 1, reg_lambda = 33.33 ,  num_leaves = 68, n_estimators = 1800, min_child_samples = 21, max_depth = 4, learning_rate = 0.136, colsample_bytree = 0.5, alpha = 17,  random_state = 0)
model.fit(trainx,trainy.values.ravel())
predictions = pd.DataFrame(model.predict(test), index = test.index, columns =['quality'])

In [ ]:
model = RandomForestClassifier()
criterion = ['entropy', 'gini']
max_depth = np.array(range(1, 100))
min_samples_split= np.array(range(1, 100))
n_estimators = np.array(range(100,300,2))
param = {'criterion': criterion, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'n_estimators': n_estimators}
eval_model(model, param, train, y, cohen_kappa)

# The best model
### Random Forest
### Randomizedsearch with nested cv
### Score of 0.590

In [ ]:
model = RandomForestClassifier(n_estimators = 122, min_samples_split = 88, max_depth = 31, criterion = 'entropy', random_state = 0)
model.fit(trainx,trainy.values.ravel())
predictions = pd.DataFrame(model.predict(test), index = test.index, columns =['quality'])


In [ ]:
predictions.to_csv('submission.csv')